In [10]:
import time
import arrow
#import delorean
from datetime import datetime
import pandas as pd

In [26]:
from enum import Enum
class Token(Enum):
    NUM=0
    PLUS=1
    MINUS=2
    DAY=3
    WEEK=4
    MONTH=5
    END=6

In [33]:


class DateTranslator:
    def __init__(self, init_date=None):
        if init_date is None:
            self.data = arrow.now()
        else:
            self.data = datetime.strptime(init_date, "%Y%m%d")

  
    def translate_date(self, date_tokens):
        #return a string with the translated date in format yyyymmdd
        for i in date_tokens:
            if i==1:
                self.data.shift(years=1)
            if i==2:
                self.data.shift(days=1) 

    def tokenizer(self, date_phrase):
        #return list of tokens
        tokens=[]
        token_map={'מחר':((Token.PLUS,0),(Token.NUM,1),(Token.DAY,0)),
                  'אתמול':((Token.MINUS,0),(Token.NUM,1),(Token.DAY,0)),
                  'היום': None,
                   'בעוד':Token.PLUS,
                  'לפני':Token.MINUS,
                  'חודשים':Token.MONTH,
                  'שבועות':Token.WEEK,
                  'ימים':Token.DAY,
                  'חודש':((Token.NUM, 2),(Token.MONTH, 0)),
                  'שבוע':((Token.NUM, 1),(Token.WEEK, 0)),
                  'יום':((Token.NUM, 2),(Token.DAY, 0)),
                  'חודשיים':((Token.NUM, 2),(Token.MONTH, 0)),
                  'שבועיים':((Token.NUM, 2),(Token.WEEK, 0)),
                  'יומיים':((Token.NUM, 2),(Token.DAY, 0)),
                  '.':Token.END}
        
        num_to_int={'אחד':1,
                  'שניים':2,
                  'שני': 2,
                   'שנים':2,
                  'שלושה':3,
                  'ארבעה':4,
                  'חמישה':5,
                  'שישה':6,
                  'שבעה':7,
                  'שמונה':8,
                  'תשעה':9,
                  'עשרה':10,
                  'עשר':10,
                  'עשרים':20,
                  'שלושים':30}
        
        parts=date_phrase[:-1].split()+['.']
        for p in parts:
            if p[0]=='ו':
                p = p[1:]
            if p in token_map:
                if type(token_map[p])==tuple:
                    tokens.extend(token_map[p])
                else:
                    tokens.append((token_map[p],0))
            else:
                tokens.append((Token.NUM, num_to_int[p]))

        return tokens
        
#         with open("date_grammar.txt",encoding='utf-8') as f:
#             for line in f:
#                 (key, val) = line.split()
#                 token_map[int(val)] = key

#         date_phrase = date_phrase[:-1].split()
#         tokens = []
#         for p in date_phrase:
#             tokens.append([key for key,val in token_map.items() if val == p][0])
#         return tokens


    def run(self, date_phrase):
        #return translated date in format yyyymmdd
        temp = self.tokenizer(date_phrase)
        #temp = self.translate_date(temp)
#         temp = temp.format('YYYYMMDD')
        return temp

In [34]:
def main():
    #driver function
    date_translator=DateTranslator('20200611')
    for phrase in data:
        print(phrase)
        print(date_translator.run(phrase))
    #20200614

main()

לפני חמישה ימים.
[(<Token.MINUS: 2>, 0), (<Token.NUM: 0>, 5), (<Token.DAY: 3>, 0), (<Token.END: 6>, 0)]
לפני יומיים.
[(<Token.MINUS: 2>, 0), (<Token.NUM: 0>, 2), (<Token.DAY: 3>, 0), (<Token.END: 6>, 0)]
בעוד שלושה שבועות.
[(<Token.PLUS: 1>, 0), (<Token.NUM: 0>, 3), (<Token.WEEK: 4>, 0), (<Token.END: 6>, 0)]
בעוד שבוע וארבעה ימים.
[(<Token.PLUS: 1>, 0), (<Token.NUM: 0>, 1), (<Token.WEEK: 4>, 0), (<Token.NUM: 0>, 4), (<Token.DAY: 3>, 0), (<Token.END: 6>, 0)]
לפני שלושה חודשים וארבעה ימים.
[(<Token.MINUS: 2>, 0), (<Token.NUM: 0>, 3), (<Token.MONTH: 5>, 0), (<Token.NUM: 0>, 4), (<Token.DAY: 3>, 0), (<Token.END: 6>, 0)]
לפני חודשיים ויום אחד.
[(<Token.MINUS: 2>, 0), (<Token.NUM: 0>, 2), (<Token.MONTH: 5>, 0), (<Token.NUM: 0>, 2), (<Token.DAY: 3>, 0), (<Token.NUM: 0>, 1), (<Token.END: 6>, 0)]


In [7]:
datetime.strptime('20200618', "%Y%m%d")

datetime.datetime(2020, 6, 18, 0, 0)

In [31]:
data = ["לפני חמישה ימים.",
"לפני יומיים.",
"בעוד שלושה שבועות.",
"בעוד שבוע וארבעה ימים.",
"לפני שלושה חודשים וארבעה ימים.",
"לפני חודשיים ויום אחד."]

In [85]:
date_tokens1 = "לפני חמישה ימים."
date_tokens2 = "לפני יומיים."
date_tokens3 = "בעוד שלושה שבועות."
date_tokens4 = "בעוד שבוע וארבעה ימים."
date_tokens5 = "לפני שלושה חודשים וארבעה ימים."
date_tokens6 = "לפני חודשיים ויום אחד."

In [24]:
date_phrase="לפני שלושה חודשים וארבעה ימים."
date_phrase.split()
(date_phrase[:-1].split()+['.'])

['לפני', 'שלושה', 'חודשים', 'וארבעה', 'ימים', '.']

In [25]:
"וארבעה"[0]

'ו'